<a id="top"></a>
# 1. ToC

In [54]:
import pandas as pd
import numpy as np

<a id="functions"></a>
# 2. Functions

## 2.1 General

In [55]:
def nan_values_of_all_columns(this_df, return_dict:[bool] = False):
    """prints (defaults) or returns dictionary of all columns in this_df and their respective nan-values"""
    if not return_dict:
        for col in this_df.columns:
            print(col, this_df[col].isna().sum())
        return
    return {col:this_df[col].isna().sum() for col in this_df.columns}

In [56]:
def return_float(x:[str] = None, repl:[str]=","):
    """replaces repl in x and returns float, if """
    return float(x.replace(",",".")) if x else None

In [57]:
def split_column_names_by(this_df, include_dtypes:[list]=["int","float"]):
    # __author__ = "Roman Kaltschew"
    """takes in this_df, splits this_df's columns into two lists of column name by dtype of columns (default ["int","float"]) 
       and returns tuple (incl_lst, excl_lst)"""
    incl_lst = [_ for _ in this_df.select_dtypes(include=include_dtypes).dtypes.index]
    excl_lst = [_ for _ in this_df.select_dtypes(exclude=include_dtypes).dtypes.index]
    return (incl_lst, excl_lst)

In [58]:
def snake_case_columns(col_names:[list]=None):
    # __author__ = "Roman Kaltschew"
    """changes all columns to snake_case (from CamelCase)"""
    if not col_names or type(col_names) != list:
        print("No list of column names provided")
        return None
    new_cols = []
    for col in col_names:
        if col.isupper():
            new_cols.append(col.lower())
        else:
            this_col = col[0].lower()
            for _ in col[1:]:
                if _.isupper():
                    this_col += f"_{_.lower()}"
                else:
                    this_col += _
            new_cols.append(this_col)
    return new_cols

## 2.2 Project specific

In [59]:
def columns_dataset_overview():
    """PROJECTSPECIFIC!!!!"""
    col_dat18 = [_ for _ in data2018.columns]
    col_dat19 = [_ for _ in data2019.columns]
    col_dat20 = [_ for _ in data2020.columns]
    print("2018:",len(col_dat18),"columns")
    print("2019:",len(col_dat19),"columns")
    print("2020:",len(col_dat20),"columns")
    
    columns_in_all = []
    for c in col_dat18:
        if c not in columns_in_all:
            if c in col_dat19 and c in col_dat20:
                columns_in_all.append(c)
    print("Number of columns identical in all datasets:", len(columns_in_all))
    print(columns_in_all)
    # ---
    print("\nMismatching columns:")
    columns_not_in_all = []
    for c in col_dat18:
        if c not in col_dat19 or c not in col_dat20:
            columns_not_in_all.append((c, 2018))
    for c in col_dat19:
        if c not in col_dat18 or c not in col_dat20:
            columns_not_in_all.append((c, 2019))
    for c in col_dat20:
        if c not in col_dat18 or c not in col_dat19:
            columns_not_in_all.append((c, 2020))
    for _ in columns_not_in_all:
        print(_)

In [60]:
def change_weekday(day:[int] = 0):
    weekdays = {
        1: "Sun",
        2: "Mon",
        3: "Tue",
        4: "Wed",
        5: "Thu",
        6: "Fri",
        7: "Sat",
    }
    if day == 0:
        return None
    return weekdays[day]

In [61]:
def bin_hours(hour:[int]):
    day_times = {
        "night": [0,1,2,3,4,5],
        "morning": [6,7,8,9,10,11],
        "afternoon": [12,13,14,15,16,17],
        "evening": [18,19,20,21,22,23],
    }
    for d_t, h in day_times.items():
        if hour in h:
            return d_t

In [62]:
def change_acc_type(cat:[int]):
    if cat == 1:
        return "killed"
    elif cat == 2:
        return "seriously_injured"
    else:
        return "slightly_injured"

In [63]:
def change_street_cond(cat:[int]):
    if cat == 1:
        return "wet"
    elif cat == 2:
        return "icy"
    else:
        return "dry"

In [64]:
def change_light_cond(cat:[int]):
    if cat == 1:
        return "dusk"
    elif cat == 2:
        return "dark"
    else:
        return "daylight"

In [65]:
def numbers_df(this_df, acc_outcome:[str]=None):
    if not acc_outcome:
        print("No accident outcome defined.")
        return None
    forms_of_transport = ['IstPKW', 'IstKrad', 'IstGkfz', 'IstRad', 'IstFuss', 'IstSonstige']
    names_of_transport = ["car", "motor bike", "truck", "bicycle", "pedestrians", "other"]

    acc_num_df = pd.DataFrame(columns=["category", "total", "percentage", "total_deaths", "death_perc"])
    acc_num_df.category = names_of_transport
    
    # print(f"Total accidents: {len(this_df)}. Total deaths: {len(this_df[this_df.accident_outcome == 'killed'])}.")
    totals = []
    to_prc = []
    to_dea = []
    de_prc = []
    for f in forms_of_transport:
        f_df = this_df[this_df[f] == 1]
        totals.append(len(f_df))
        to_prc.append(round(len(f_df)*100/len(this_df),2))
        to_dea.append(len(f_df[f_df.accident_outcome == acc_outcome]))
        de_prc.append(round(len(f_df[f_df.accident_outcome == acc_outcome])*100/len(this_df[this_df.accident_outcome == acc_outcome]),2))
    df_ = pd.DataFrame
    acc_num_df.total = totals
    acc_num_df.percentage = to_prc
    acc_num_df.total_deaths = to_dea
    acc_num_df.death_perc = de_prc
    return acc_num_df

In [1]:
len(data_)

NameError: name 'data_' is not defined

<a id = "combine_data"></a>
# 3. Creating combined dataset

## 3.1 Reading all files and basic column name cleaning

In [66]:
data2018 = pd.read_csv(r"data\AfSBBB_BE_LOR_Strasse_Strassenverkehrsunfaelle_2018_Datensatz.csv", encoding="latin-1", sep=";")
data2019 = pd.read_csv(r"data\AfSBBB_BE_LOR_Strasse_Strassenverkehrsunfaelle_2019_Datensatz.csv", encoding="latin-1", sep=";")
data2020 = pd.read_csv(r"data\AfSBBB_BE_LOR_Strasse_Strassenverkehrsunfaelle_2020_Datensatz.csv", encoding="latin-1", sep=";")

In [67]:
# calling a function specific to this project
columns_dataset_overview()

2018: 25 columns
2019: 24 columns
2020: 24 columns
Number of columns identical in all datasets: 21
['OBJECTID', 'LAND', 'BEZ', 'LOR', 'UJAHR', 'UMONAT', 'USTUNDE', 'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1', 'ULICHTVERH', 'IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz', 'LINREFX', 'LINREFY', 'XGCSWGS84', 'YGCSWGS84']

Mismatching columns:
('STRASSE', 2018)
('LOR_ab_2021', 2018)
('IstSonstig', 2018)
('STRZUSTAND', 2018)
('STRASSE', 2019)
('IstSonstige', 2019)
('USTRZUSTAND', 2019)
('LOR_ab_2021', 2020)
('IstSonstige', 2020)
('USTRZUSTAND', 2020)


In [68]:
data2018.rename(columns={'IstSonstig': 'IstSonstige', 'STRZUSTAND': 'USTRZUSTAND'}, inplace=True)

In [69]:
# reordering all columns and drop superfluous columns
data2018 = data2018[['UJAHR', 'UMONAT', 'USTUNDE', 'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1', 'ULICHTVERH', 'IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz', 'IstSonstige', 'USTRZUSTAND', 'LINREFX', 'LINREFY', 'XGCSWGS84', 'YGCSWGS84']]
data2019 = data2019[['UJAHR', 'UMONAT', 'USTUNDE', 'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1', 'ULICHTVERH', 'IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz', 'IstSonstige', 'USTRZUSTAND', 'LINREFX', 'LINREFY', 'XGCSWGS84', 'YGCSWGS84']]
data2020 = data2020[['UJAHR', 'UMONAT', 'USTUNDE', 'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1', 'ULICHTVERH', 'IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz', 'IstSonstige', 'USTRZUSTAND', 'LINREFX', 'LINREFY', 'XGCSWGS84', 'YGCSWGS84']]

In [70]:
# concat
data_ = pd.concat([data2018, data2019, data2020])

In [71]:
data_.columns = snake_case_columns(list(data_.columns))
data_.rename(columns={'ist_p_k_w': 'ist_pkw'}, inplace=True)


In [72]:
# remove outlier in ist_sonstige
data_ = data_[data_.ist_sonstige != "812083,7893"] 
# set type to int
data_.ist_sonstige = data_.ist_sonstige.astype("int64")


## 3.2 Feature Engineering 

In [73]:
# usable coordinates of the dataset
data_["long"] = list(map(return_float, data_["ygcswgs84"]))
data_["lat"] = list(map(return_float, data_["xgcswgs84"]))

In [74]:
data_.drop(['linrefx', 'linrefy', 'xgcswgs84', 'ygcswgs84'], axis=1, inplace=True)

In [75]:
# weekday column
data_["weekday"] = list(map(change_weekday, data_["uwochentag"]))
# data_.drop("uwochentag", axis=1, inplace=True)

# accident outcome
data_["accident_outcome"] = list(map(change_acc_type, data_["ukategorie"]))
# data_.drop("ukategorie", axis=1, inplace=True)

# time_of_day
data_["time_of_day"] = list(map(bin_hours, data_["ustunde"]))
# data_.drop("ustunde", axis=1, inplace=True)

# time_of_day
data_["street_condition"] = list(map(change_street_cond, data_["ustrzustand"]))
# data_.drop("ustrzustand", axis=1, inplace=True)

# time_of_day
data_["light_condition"] = list(map(change_light_cond, data_["ulichtverh"]))
# data_.drop("ulichtverh", axis=1, inplace=True)


In [76]:
data_

,ujahr,umonat,ustunde,uwochentag,ukategorie,uart,utyp1,ulichtverh,ist_rad,ist_pkw,...,ist_gkfz,ist_sonstige,ustrzustand,long,lat,weekday,accident_outcome,time_of_day,street_condition,light_condition
0,2018,1,15,4,3,6,4,0,0,1,...,0,0,1,52.513597,13.475018,Wed,slightly_injured,afternoon,wet,daylight
1,2018,1,11,2,3,2,6,0,0,1,...,0,0,0,52.587259,13.291022,Mon,slightly_injured,morning,dry,daylight
2,2018,1,9,3,3,6,4,0,0,1,...,0,0,0,52.526019,13.420578,Tue,slightly_injured,morning,dry,daylight
3,2018,1,17,2,3,6,7,2,0,1,...,0,0,0,52.481844,13.348288,Mon,slightly_injured,afternoon,dry,dark
4,2018,1,15,4,3,6,7,1,1,0,...,0,0,1,52.583472,13.403228,Wed,slightly_injured,afternoon,wet,dusk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11805,2020,12,7,4,3,6,2,1,0,1,...,0,0,1,52.466442,13.377941,Wed,slightly_injured,morning,wet,dusk
11806,2020,12,17,3,3,5,3,2,0,1,...,0,0,1,52.613018,13.310915,Tue,slightly_injured,afternoon,wet,dark
11807,2020,12,14,3,3,5,3,0,0,1,...,0,1,0,52.506091,13.419649,Tue,slightly_injured,afternoon,dry,daylight
11808,2020,12,16,3,3,6,2,2,0,1,...,0,0,1,52.510570,13.391817,Tue,slightly_injured,afternoon,wet,dark


In [77]:
for _ in list(data_.columns):
    print(_, end=", ")

ujahr, umonat, ustunde, uwochentag, ukategorie, uart, utyp1, ulichtverh, ist_rad, ist_pkw, ist_fuss, ist_krad, ist_gkfz, ist_sonstige, ustrzustand, long, lat, weekday, accident_outcome, time_of_day, street_condition, light_condition, 

In [80]:
data_ = data_[["ujahr", "umonat", "weekday", "uwochentag", "long", "lat", "time_of_day", "ustunde", "light_condition", 
               "ulichtverh", "street_condition", "ustrzustand", "uart", "utyp1", "ist_rad", "ist_pkw", "ist_fuss", "ist_krad", 
               "ist_gkfz", "ist_sonstige", "ukategorie", "accident_outcome"]]

## 3.3 Save to csv (if needed)

In [82]:
# data_.to_csv("./data/cleaned_data.csv", index=False)

In [2]:
len(data_)

NameError: name 'data_' is not defined